In [4]:
#pip install face_recognition
!pip install cmake boost dlib face_recognition


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 26.7 MB 7.2 MB/s eta 0:00:01     |████████████████████████▉       | 20.7 MB 7.2 MB/s eta 0:00:01     |█████████████████████████████   | 24.2 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.5 MB/s  eta 0:00:01
  Created wheel for boost: filename=boost-0.1-py3-none-any.whl size=12460 sha256=1ee2a7c4a29a65f2eaf76c9a33d6fcb483e46882953e83399e498c431f7eb6b6
  Stored in directory: /home/znebieri/.cache/pip/wheels/ba/d3/ce/40a4eb92e7aad98445295e20e931fb10b09943c453b37b570c
Successfully built boost
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import os
import face_recognition
import random

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for subdir, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.jpeg', '.jpg')):
                img_path = os.path.join(subdir, file)
                images.append(img_path)
    return images

# Function to detect faces in an image
def detect_faces(image_path, model="cnn"):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image, model=model)
    return len(face_locations) > 0

def test_model(human_faces, non_human_faces, percentage, model="cnn"):
    # Calculate the number of images to use based on the percentage
    num_human_faces = int(len(human_faces) * percentage / 100)
    num_non_human_faces = int(len(non_human_faces) * percentage / 100)

    # Randomly select images to use
    human_faces_sample = random.sample(human_faces, num_human_faces)
    non_human_faces_sample = random.sample(non_human_faces, num_non_human_faces)

    human_correct = sum(detect_faces(img, model=model) for img in human_faces_sample)
    non_human_correct = sum(not detect_faces(img, model=model) for img in non_human_faces_sample)

    human_accuracy = human_correct / num_human_faces * 100
    non_human_accuracy = non_human_correct / num_non_human_faces * 100
    total_accuracy = (human_correct + non_human_correct) / (num_human_faces + num_non_human_faces) * 100

    print(f"Tested {num_human_faces} human face images.")
    print(f"Tested {num_non_human_faces} non-human face images.")
    print(f"Human face classification accuracy: {human_accuracy:.2f}%")
    print(f"Non-human face classification accuracy: {non_human_accuracy:.2f}%")
    print(f"Total classification accuracy: {total_accuracy:.2f}%")

# Load images
human_faces_folder = 'temp_human_faces'
non_human_faces_folder = 'temp_non_human_faces'

human_faces = load_images_from_folder(human_faces_folder)
non_human_faces = load_images_from_folder(non_human_faces_folder)

# Define the percentage of images to use for testing
percentage = 5  

# Test the model
test_model(human_faces, non_human_faces, percentage, model="cnn")


Tested 73 human face images.
Tested 65 non-human face images.
Human face classification accuracy: 100.00%
Non-human face classification accuracy: 98.46%
Total classification accuracy: 99.28%


In [4]:
import os
import zipfile
import pandas as pd
import shutil
import face_recognition
import random
import torch

if torch.cuda.is_available():
    torch.device('cuda')
    print("Using cuda")
else:
    torch.device('cpu')
    print("Using cpu")
    
# Function to extract zip files
def extract_zip(file_path, extract_to):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for subdir, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.jpeg', '.jpg')):
                img_path = os.path.join(subdir, file)
                images.append(img_path)
    return images

# Function to detect faces in an image
def detect_faces(image_path, model="cnn"):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image, model=model)
    return len(face_locations) > 0

# Function to test the model
def test_model(human_faces, percentage, model="cnn"):
    # Calculate the number of images to use based on the percentage
    num_human_faces = int(len(human_faces) * percentage / 100)

    # Randomly select images to use
    human_faces_sample = random.sample(human_faces, num_human_faces)

    human_correct = sum(detect_faces(img, model=model) for img in human_faces_sample)

    human_accuracy = human_correct / num_human_faces * 100

    print(f"Tested {num_human_faces} human face images.")
    print(f"Human face classification accuracy: {human_accuracy:.2f}%")

    return human_accuracy

# Path to the main zipped folder
main_zip_path = 'Adversal_Images.zip'
extract_main_folder = 'Adversal_Images_extracted'
extract_zip(main_zip_path, extract_main_folder)

results = []

# Iterate through each zipped folder inside the main extracted folder
for file in os.listdir(extract_main_folder):
    if file.endswith('.zip'):
        zip_path = os.path.join(extract_main_folder, file)
        extract_folder = os.path.join(extract_main_folder, file.replace('.zip', ''))
        
        # Extract the zipped subfolder
        extract_zip(zip_path, extract_folder)
        
        # Get the number associated with the folder
        folder_number = file.split('_')[-1].replace('.zip', '')
        print(f'Evaluating folder number: {folder_number}')
        
        # Load images
        human_faces = load_images_from_folder(extract_folder)
        
        # Define the percentage of images to use for testing
        percentage = 50  
        
        # Test the model
        human_accuracy = test_model(human_faces, percentage, model="cnn")
        
        # Append the results to the list
        results.append({
            'folder_number': folder_number,
            'human_accuracy': human_accuracy
        })
        
        shutil.rmtree(extract_folder)

# Convert the results to a pandas DataFrame
results_df = pd.DataFrame(results)
results_df


Using cuda
Evaluating folder number: 0.25
Tested 170 human face images.
Human face classification accuracy: 55.88%
Evaluating folder number: 0.06
Tested 170 human face images.
Human face classification accuracy: 100.00%
Evaluating folder number: 0.01
Tested 170 human face images.
Human face classification accuracy: 100.00%
Evaluating folder number: 0.4
Tested 170 human face images.
Human face classification accuracy: 2.94%
Evaluating folder number: 0.3
Tested 170 human face images.
Human face classification accuracy: 28.24%
Evaluating folder number: 0.16
Tested 170 human face images.
Human face classification accuracy: 95.88%
Evaluating folder number: 0.35
Tested 170 human face images.
Human face classification accuracy: 6.47%
Evaluating folder number: 0.09
Tested 170 human face images.
Human face classification accuracy: 100.00%
Evaluating folder number: 0.2
Tested 170 human face images.
Human face classification accuracy: 83.53%
Evaluating folder number: 0.5
Tested 170 human face ima

,folder_number,human_accuracy
0,0.25,55.882353
1,0.06,100.000000
2,0.01,100.000000
3,0.4,2.941176
4,0.3,28.235294
5,0.16,95.882353
6,0.35,6.470588
7,0.09,100.000000
8,0.2,83.529412
9,0.5,0.000000
